# Given the elmo embeddings hdf5, generate the cross val pickled files

In [18]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
%autoreload

import os, random, pandas as pd, numpy as np
from sklearn.model_selection import StratifiedKFold
import pickle
import sys
sys.path.append('../..')
import relation_extraction.data.utils as utils
import itertools
# num folds are 10 for semeval but 5 for ddi and i2b2
# np.random.seed(SEED)
# random.seed(SEED)
#TODO: replace preprocessing type with the type of preprocessing you are doing
dataset = 'ddi' #TODO: update this
num_folds = 5 #TODO: update this
preprocessing_type = 'original' #TODO: update this: in most cases, it will be original
RESOURCE_PATH = "/crimea/geeticka/data/relation_extraction/" + dataset + "/pre-processed/" + preprocessing_type + "/"
def res(path): return os.path.join(RESOURCE_PATH, path)
if not os.path.exists(res('elmo/pickled-files/')):
    os.makedirs(res('elmo/pickled-files/'))

In [20]:
def get_dataframe(splitted_data, elmo_data):
    # now need to figure out how to create the csv file
    sen = splitted_data[0]
    rel = splitted_data[1]
    e1 = splitted_data[2]
    e2 = splitted_data[3]
    elmo_data = elmo_data[0]
    data = pd.DataFrame({'sentences': sen, 'relations': rel, 'e1_pos': e1, 'e2_pos': e2, \
                         'elmo_embeddings': elmo_data}, \
                        columns=['sentences', 'relations', 'e1_pos', 'e2_pos', 'elmo_embeddings'])
    return data

In [21]:
def pickle_data(data, filename1, filename2, SEED, num_folds, border_size):
    np.random.seed(SEED)
    random.seed(SEED)
    print("seed is", SEED)
    N = len(data)
    K = num_folds

    splitter = StratifiedKFold(n_splits=K, shuffle=True, random_state=SEED)
    fold_indices = [indices for _, indices in splitter.split(
        data[['sentences', 'e1_pos', 'e2_pos', 'elmo_embeddings']].values,
        data['relations'].values
    )]

    assert len(fold_indices) == K, "Incorrect number of folds."
    assert len(np.concatenate(fold_indices)) == N, "Folds not comprehensive."

    splits = []
    for fold in range(K):
        test_fold  = fold_indices[fold]
        dev_fold   = fold_indices[(fold + 1) % K]

        non_train_start = fold
        non_train_end   = (fold + 2) % K

        if non_train_start < non_train_end:
            train_fold = np.concatenate(fold_indices[non_train_end:] + fold_indices[:non_train_start])
        else: train_fold = np.concatenate(fold_indices[non_train_end:non_train_start])
        joined = np.concatenate((train_fold, dev_fold, test_fold))

        assert len(joined) == N, (
            "Split not comprehensive for fold {fold}:\n"
            "  len(train_fold): {train}\n"
            "  len(dev_fold): {dev}\n"
            "  len(test_fold): {test}\n"
            "  len(joined): {joined}\n"
            "  N: {N}".format(
                N=N, fold=fold, train=len(train_fold), dev=len(dev_fold), test=len(test_fold), joined=len(joined)
            )
        )

        splits.append((
            data.iloc[train_fold],
            data.iloc[dev_fold],
            data.iloc[test_fold],
        ))

    assert len(splits) == K, "Too few splits"

    with open(res('elmo/pickled-files/'+filename1.format(seed=SEED, K=K, bordersize=str(border_size))), mode='wb') as f: pickle.dump(splits, f)
    with open(res('elmo/pickled-files/'+filename2.format(seed=SEED, K=K, bordersize=str(border_size))), mode='wb') as f: pickle.dump(splits, f, protocol=2)

In [22]:
# border_sizes = [-1, 20, 50, 1]
border_sizes = [-1]

In [ ]:
for border_size in border_sizes:
    train_data = open(res('train_' + preprocessing_type + '.txt'))
    splitted_data = utils.split_data_cut_sentence(train_data, border_size=border_size)
    elmo_embeddings = utils.get_elmo_embeddings(res('elmo/train_' + preprocessing_type + '_border_' + str(border_size) + '.hdf5'))
    data = get_dataframe(splitted_data, elmo_embeddings)
    pickle_data(data, 'seed_{seed}_{K}-fold-border_{bordersize}_' + preprocessing_type + '.pkl', 
                'seed_{seed}_{K}-fold-border_{bordersize}_' + preprocessing_type + '_py2.pkl', 5, 
                num_folds, border_size)

In [17]:
# seeds = [1, 2, 3, 4]

In [18]:
# for seed in seeds:
#     train_data = open(res('train_original.txt'))
#     splitted_data = utils.split_data_cut_sentence(train_data, border_size=50)
#     elmo_embeddings = utils.get_elmo_embeddings(res('elmo/train_original_border_50.hdf5'))
#     data = get_dataframe(splitted_data, elmo_embeddings)
#     pickle_data(data, 'seed_{seed}_{K}-fold-border_{bordersize}_original.pkl', 
#                 'seed_{seed}_{K}-fold-border_{bordersize}_original_py2.pkl', seed, num_folds, 50)

seed is 1
seed is 2
seed is 3
seed is 4
